This file trains a Boosted Decision Tree to classify between a quark and a gulon-initiated jet.

In [ ]:
import ROOT
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.callback import EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

fileName = "/eos/user/k/kmonagha/dijet_events_data/dijet_events_full_2001-2181.root"
treeName = "dijets"
d = ROOT.RDataFrame(treeName, fileName)

#Add branch for jet girth variable
ROOT.gInterpreter.Declare("""
float ComputeJetGirth(
    const ROOT::VecOps::RVec<float>& px,
    const ROOT::VecOps::RVec<float>& py,
    const ROOT::VecOps::RVec<float>& pz,
    const ROOT::VecOps::RVec<float>& E,
    float jet_eta, float jet_phi, float jet_pt
) {
    float girth = 0.0;
    for (size_t i = 0; i < px.size(); ++i) {
        float pt = std::sqrt(px[i]*px[i] + py[i]*py[i]);
        float p = std::sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
        float eta = 0.5 * std::log((E[i] + pz[i]) / (E[i] - pz[i] + 1e-8));
        float phi = std::atan2(py[i], px[i]);
        
        float delta_eta = eta - jet_eta;
        float delta_phi = std::atan2(std::sin(phi - jet_phi), std::cos(phi - jet_phi));  // Wrap phi correctly
        float delta_R = std::sqrt(delta_eta * delta_eta + delta_phi * delta_phi);
        
        girth += pt * delta_R;
    }
    return girth / (jet_pt + 1e-8);  // Normalize by jet pt
}
""")

d = d.Define("jet1_girth", "ComputeJetGirth(jet1_Cons_Px, jet1_Cons_Py, jet1_Cons_Pz, jet1_Cons_E, jet1_eta, jet1_phi, jet1_pt)")

In [ ]:
#Extract the momentum px,py,pz, along with energy and ID from the top 10 PT jets
ROOT.gInterpreter.Declare("""
std::vector<float> TopNConstituentFeature(
    const ROOT::VecOps::RVec<float>& px,
    const ROOT::VecOps::RVec<float>& py,
    const ROOT::VecOps::RVec<float>& pz,
    const ROOT::VecOps::RVec<float>& E,
    const ROOT::VecOps::RVec<int>& id,
    int feature_idx // 0: px, 1: py, 2: pz, 3: E, 4: ID
) {
    std::vector<std::tuple<float, float, float, float, int, float>> constituents;

    for (size_t i = 0; i < px.size(); ++i) {
        float pt = std::sqrt(px[i]*px[i] + py[i]*py[i]);
        constituents.emplace_back(px[i], py[i], pz[i], E[i], id[i], pt);
    }

    std::sort(constituents.begin(), constituents.end(), 
        [](const auto& a, const auto& b) {
            return std::get<5>(a) > std::get<5>(b); // sort by pt descending
        });

    std::vector<float> top_features;
    for (int i = 0; i < 10; ++i) {
        if (i < constituents.size()) {
            switch (feature_idx) {
                case 0: top_features.push_back(std::get<0>(constituents[i])); break;
                case 1: top_features.push_back(std::get<1>(constituents[i])); break;
                case 2: top_features.push_back(std::get<2>(constituents[i])); break;
                case 3: top_features.push_back(std::get<3>(constituents[i])); break;
                case 4: top_features.push_back(static_cast<float>(std::get<4>(constituents[i]))); break;
                default: top_features.push_back(0.0); break;
            }
        } else {
            top_features.push_back(0.0); // pad with 0s if fewer than 10 constituents
        }
    }

    return top_features;
}
""")

for i, name in enumerate(["Px", "Py", "Pz", "E", "ID"]):
    d = d.Define(f"Top10_{name}", f"TopNConstituentFeature(jet1_Cons_Px, jet1_Cons_Py, jet1_Cons_Pz, jet1_Cons_E, jet1_Cons_ID, {i})")
    for j in range(10):
        d = d.Define(f"Cons_{name.lower()}_{j+1}", f"Top10_{name}[{j}]")

Cons_e_1
Cons_e_10
Cons_e_2
Cons_e_3
Cons_e_4
Cons_e_5
Cons_e_6
Cons_e_7
Cons_e_8
Cons_e_9
Cons_id_1
Cons_id_10
Cons_id_2
Cons_id_3
Cons_id_4
Cons_id_5
Cons_id_6
Cons_id_7
Cons_id_8
Cons_id_9
Cons_px_1
Cons_px_10
Cons_px_2
Cons_px_3
Cons_px_4
Cons_px_5
Cons_px_6
Cons_px_7
Cons_px_8
Cons_px_9
Cons_py_1
Cons_py_10
Cons_py_2
Cons_py_3
Cons_py_4
Cons_py_5
Cons_py_6
Cons_py_7
Cons_py_8
Cons_py_9
Cons_pz_1
Cons_pz_10
Cons_pz_2
Cons_pz_3
Cons_pz_4
Cons_pz_5
Cons_pz_6
Cons_pz_7
Cons_pz_8
Cons_pz_9
DeltaEta
DeltaPhi
DeltaY
Mjj
Q2
Top10_E
Top10_ID
Top10_Px
Top10_Py
Top10_Pz
chi
fCoordinates
fCoordinates.fE
fCoordinates.fEta
fCoordinates.fPhi
fCoordinates.fPt
id1
id2
jet1_Cons_E
jet1_Cons_ID
jet1_Cons_Px
jet1_Cons_Py
jet1_Cons_Pz
jet1_E
jet1_eta
jet1_girth
jet1_mass
jet1_nChargedCons
jet1_nCons
jet1_partonID
jet1_phi
jet1_pt
jet1_px
jet1_py
jet1_pz
jet1_vec
jet1_vec.fCoordinates
jet1_vec.fCoordinates.fE
jet1_vec.fCoordinates.fEta
jet1_vec.fCoordinates.fPhi
jet1_vec.fCoordinates.fPt
jet2_Cons_E
je

In [ ]:
# Select columns for training

extra_features = [f"Cons_{f}_{i}" for f in ["px", "py", "pz", "e", "id"] for i in range(1, 11)]

columns = [
    "jet1_pt", "jet1_eta", "jet1_mass",
     "jet1_partonID", "jet1_nCons","jet1_nChargedCons","jet1_Cons_ID","jet1_girth"
] + extra_features

# Convert to pandas DataFrame
df = d.AsNumpy(columns)
df = pd.DataFrame(df)

# --- Prepare training data ---
def parton_label(pid):
    if abs(pid) in [1, 2, 3, 4, 5]:
        return 1  # quark
    elif pid == 21:
        return 0  # gluon
    else:
        return -1

df["label"] = df["jet1_partonID"].apply(parton_label)
df = df[df["label"] >= 0]  # filter out unknowns so only left with 1 or 0 (quark or gluon) 

features = [
    "jet1_pt", "jet1_eta", "jet1_mass", "jet1_nCons",
    "jet1_nChargedCons","jet1_girth"
] + extra_features

X = df[features]
y = df["label"] #target values i.e, the real identification 'output' pertaining to the x input


           jet1_pt  jet1_eta  jet1_mass  jet1_partonID  jet1_nCons  \
0        30.219925  4.681803   6.193370             21          17   
1        24.678329  4.142051   6.231205             21          22   
2        26.016535  2.678441  10.137779              2          14   
3        24.702797  3.567970   7.030400              1          20   
4        29.058043  4.753375   7.327851              1          15   
...            ...       ...        ...            ...         ...   
6763840  32.164856  2.243918   9.829671             21          14   
6763841  40.094532  2.958403   8.911090             21          20   
6763843  42.528320  4.440619  10.614146             21          19   
6763844  51.821274  2.343277  13.961857              2          17   
6763845  47.947353  3.445249  11.001780              2          18   

         jet1_nChargedCons                                       jet1_Cons_ID  \
0                       10  [22, 22, -211, 211, 22, -321, 22, -321, -211, ... 

In [ ]:
df.to_pickle("/eos/user/k/kmonagha/BDT_models/full_dataframe_debug.pkl") #saving dataframe to be accessed in quark_gluon_BDT_plots Notebook

In [ ]:
model = xgb.XGBClassifier(
    n_estimators=2000,
    max_depth=8,
    learning_rate=0.05,
    early_stopping_rounds=50,
    verbose=True,
    eval_metric="logloss"
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)]
)


#--- Evaluation ---
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1] #gets the probability of each event being class 1 (quark)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

model.save_model("/eos/user/k/kmonagha/BDT_models/xgb_jet_classifier_5.json")
# to access
# loaded_model = xgb.XGBClassifier()
# loaded_model.load_model("xgb_jet_classifier.json")

In [ ]:
np.save("X_test_3.npy", X_test)
np.save("y_test_3.npy", y_test)
np.save("X_train_3.npy", X_train)
np.save("y_train_3.npy", y_train)